# Courses Demo
This Jupyter notebook is for exploring the data set courses20-21.json
which consists of all Brandeis courses in the 20-21 academic year (Fall20, Spr21, Sum21) 
which had at least 1 student enrolled.

First we need to read the json file into a list of Python dictionaries

In [1]:
import json

In [2]:
with open("pa1_data/courses20-21.json","r",encoding='utf-8') as jsonfile:
    courses = json.load(jsonfile)

## Structure of a course
Next we look at the fields of each course dictionary and their values

In [3]:
print('there are',len(courses),'courses in the dataset')
print('here is the data for course 1246')
courses[1246]

there are 7813 courses in the dataset
here is the data for course 1246


{'limit': 28,
 'times': [{'start': 1080, 'end': 1170, 'days': ['w', 'm']}],
 'enrolled': 4,
 'details': 'Instruction for this course will be offered remotely. Meeting times for this course are listed in the schedule of classes (in ET).',
 'type': 'section',
 'status_text': 'Open',
 'section': '1',
 'waiting': 0,
 'instructor': ['An', 'Huang', 'anhuang@brandeis.edu'],
 'coinstructors': [],
 'code': ['MATH', '223A'],
 'subject': 'MATH',
 'coursenum': '223A',
 'name': 'Lie Algebras: Representation Theory',
 'independent_study': False,
 'term': '1203',
 'description': "Theorems of Engel and Lie. Semisimple Lie algebras, Cartan's criterion. Universal enveloping algebras, PBW theorem, Serre's construction. Representation theory. Other topics as time permits. Usually offered every second year.\nAn Huang"}

## Cleaning the data
If we want to sort courses by instructor or by code, we need to replace the lists with tuples (which are immutable lists)

In [4]:
for course in courses:
        course['instructor'] = tuple(course['instructor'])
        course['coinstructors'] = tuple([tuple(f) for f in course['coinstructors']])
        course['code']= tuple(course['code'])
        course["enrolled"]=int(course["enrolled"])

In [5]:
print('notice that the instructor and code are tuples now')
courses[1246]

notice that the instructor and code are tuples now


{'limit': 28,
 'times': [{'start': 1080, 'end': 1170, 'days': ['w', 'm']}],
 'enrolled': 4,
 'details': 'Instruction for this course will be offered remotely. Meeting times for this course are listed in the schedule of classes (in ET).',
 'type': 'section',
 'status_text': 'Open',
 'section': '1',
 'waiting': 0,
 'instructor': ('An', 'Huang', 'anhuang@brandeis.edu'),
 'coinstructors': (),
 'code': ('MATH', '223A'),
 'subject': 'MATH',
 'coursenum': '223A',
 'name': 'Lie Algebras: Representation Theory',
 'independent_study': False,
 'term': '1203',
 'description': "Theorems of Engel and Lie. Semisimple Lie algebras, Cartan's criterion. Universal enveloping algebras, PBW theorem, Serre's construction. Representation theory. Other topics as time permits. Usually offered every second year.\nAn Huang"}

# Exploring the data set
Now we will show how to use straight python to explore the data set and answer some interesting questions. Next week we will start learning Pandas/Numpy which are packages that make it easier to explore large dataset efficiently.

Here are some questions we can try to answer:
* what are all of the subjects of courses (e.g. COSI, MATH, JAPN, PHIL, ...)
* which terms are represented?
* how many instructors taught at Brandeis last year?
* what were the five largest course sections?
* what were the five largest courses (where we combine sections)?
* which are the five largest subjects measured by number of courses offered?
* which are the five largest courses measured by number of students taught?
* which course had the most sections taught in 20-21?
* who are the top five faculty in terms of number of students taught?
* etc.

# Student Code Below Here:

A) How many faculty taught COSI courses last year?

In [6]:
instructors=set(course["instructor"] for course in courses)
print(len(instructors))

904


B) What is the total number of students taking COSI courses last year?

In [7]:
students= sum([course["enrolled"] for course in courses if course["subject"].lower()=="cosi"])
print(students)

2223


C) What was the median size of a COSI course last year (counting only those courses with at least 10 students)?

In [8]:
eligibleCourses=sorted([course ["enrolled"] for course in courses if course["enrolled"]>=10])
median = eligibleCourses[int(len(eligibleCourses)/2)]
print(median)

20


D) Create a list of tuples (E,S) where S is a subject and E is the number of students enrolled in courses in that subject, sort it and print the top 10. This shows the top 10 subjects in terms of number of students taught.

In [9]:
subjects=set(course["subject"] for course in courses)
top_courses=[tuple((sum([course["enrolled"] for course in courses if course["subject"]==subject]),subject)) for subject in subjects]
top_courses=sorted(top_courses,reverse=True )
print(top_courses[:10])


[(5318, 'HS'), (3085, 'BIOL'), (2766, 'BUS'), (2734, 'HWL'), (2322, 'CHEM'), (2315, 'ECON'), (2223, 'COSI'), (1785, 'MATH'), (1704, 'PSYC'), (1144, 'ANTH')]


These numbers seem high, but it might be because students take multiple courses in a subject at a time.

E) Do the same as in (d) but print the top 10 subjects in terms of number of courses offered.

In [10]:
subjects=set(course["subject"] for course in courses)
top_courses=[tuple((len([course["enrolled"] for course in courses if course["subject"]==subject]),subject)) for subject in subjects]
top_courses=sorted(top_courses, reverse=True)
print(top_courses[:10])

[(613, 'BIOL'), (498, 'HIST'), (417, 'PSYC'), (403, 'NEUR'), (296, 'BCHM'), (288, 'PHYS'), (274, 'HS'), (272, 'COSI'), (266, 'MUS'), (265, 'ENG')]


F) Do the same as (d) but print the top 10 subjects in terms of number of faculty teaching courses in that subject.

In [11]:
subjects=set(course["subject"] for course in courses)
top_courses=[tuple((len(set(course["instructor"] for course in courses if course["subject"]==subject)),subject)) for subject in subjects]
top_courses=sorted(top_courses, reverse=True)
print(top_courses[:10])

[(87, 'HS'), (67, 'BIOL'), (52, 'ECON'), (49, 'BCHM'), (47, 'HIST'), (47, 'BUS'), (46, 'BCBP'), (42, 'HWL'), (37, 'NEJS'), (37, 'MATH')]


G) List the top 20 faculty in terms of number of students they taught.

In [12]:
instructors=set(course["instructor"] for course in courses)
top_instructors=[(sum(course["enrolled"] for course in courses if course["instructor"]==instructor),(instructor[0]+" "+instructor[1]))for instructor in instructors]
top_instructors=sorted(top_instructors,reverse=True)[:20]
print([name[1] for name in top_instructors])

['Leah Berkenwald', 'Kene Nathan Piasta', 'Stephanie Murray', 'Milos Dolnik', 'Maria de Boef Miara', 'Bryan Ingoglia', 'Rachel V.E. Woodruff', 'Timothy J Hickey', 'Daniel Breen', 'Melissa Kosinski-Collins', 'Claudia Novack', 'Antonella DiLillo', 'Jon Chilingerian', 'Ahmad Namini', 'Iraklis Tsekourakis', 'Geoffrey Clarke', 'Peter Mistark', 'Brenda Anderson', 'Colleen Hitchcock', 'Scott A. Redenius']


H) List the top 20 courses in terms of number of students taking that course (where you combine different sections and semesters, i.e. just use the subject and course number).

In [23]:
course_set=set(course["code"] for course in courses)
top_courses2=[(sum([course["enrolled"] for course in courses if course["code"]==code]),code[0]+" "+code[1]) for code in course_set]
top_courses2=sorted(top_courses2, reverse=True)
print([topten[1] for topten in top_courses2][:20])

['HWL 1', 'HWL 1-PRE', 'BIOL 14A', 'COSI 10A', 'PSYC 10A', 'BIOL 15B', 'MATH 10A', 'BIOL 18B', 'BIOL 18A', 'CHEM 29A', 'CHEM 29B', 'CHEM 25A', 'PSYC 51A', 'CHEM 25B', 'COSI 12B', 'BUS 6A', 'CHEM 18A', 'ECON 10A', 'MATH 15A', 'COSI 21A', 'ANTH 1A', 'CHEM 11A', 'BUS 10A', 'BIOL 16A', 'CHEM 18B', 'BIOL 42A', 'CHEM 11B', 'SOC 1A', 'ECON 20A', 'HS 104B', 'EL 35A', 'COSI 131A', 'ECON 80A', 'ECON 83A', 'ECON 184B', 'COSI 164A', 'BUS 120A', 'ECON 82B', 'ECON 2A', 'LGLS 10A', 'BUS 71A', 'ENVS 2A', 'PSYC 52A', 'HISP 32A', 'HWL 41', 'HSSP 100B', 'BUS 152A', 'NPSY 199A', 'MATH 10B', 'POL 15A', 'BUS 215F', 'PHYS 10A', 'MATH 20A', 'BUS 297A', 'PSYC 34B', 'PHYS 18A', 'BUS 224G', 'PHYS 10B', 'HSSP 110A', 'PHYS 18B', 'HWL 40', 'ECON 28B', 'PSYC 32A', 'BCHM 100A', 'BUS 211F', 'IGS 10A', 'FIN 201A', 'PHIL 6A', 'BIOL 350A', 'BCHM 88B', 'COSI 29A', 'PHIL 1A', 'BUS 216F', 'PSYC 33A', 'PSYC 31A', 'MATH 23B', 'HS 491G', 'BIOL 51A', 'WGS 5A', 'SOC 83A', 'HS 489B', 'BIOL 43B', 'BIOL 350B', 'SOC 84A', 'HS 801G',

I) Create your own interesting question (each team member creates their own) and use Python to answer that question.

Ben: What are the 10 professors who teach the most COSI courses and the number of courses they taught?

In [15]:
instructors=set(course["instructor"] for course in courses if course["subject"]=="COSI")
professors_courses=[(len([course["code"] for course in courses if course["instructor"]==instructor]),instructor[0]+" "+instructor[1]) for instructor in instructors]
professors_courses=sorted(professors_courses,reverse=True)
print(professors_courses[0:10])

    
        

[(36, 'Paul Miller'), (31, 'James Pustejovsky'), (21, 'Timothy J Hickey'), (21, 'Pengyu Hong'), (18, 'Ralph Salas'), (16, 'Jordan Pollack'), (15, 'Richard Alterman'), (15, 'Olga Papaemmanouil'), (15, 'Nianwen Xue'), (15, 'Mitch Cherniack')]


Jason:

Kayla:

Kevin:

Mason: 